In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-08-10"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
214,2024-08-10,Mundo Jogos Olímpicos,06:00,Alemanha,Sérvia,2.16,1.74,169.5,1.83,1.95,1.5,1.96,1.58,6NrRQHe4,0.462963,0.574713,0.546448,0.512821,0.037676,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.152300,0.044896,0.151808,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
215,2024-08-10,Austrália Nbl1 Leste,04:30,Canberra,Maitland M.,1.78,1.94,171.5,1.86,1.84,-2.5,2.03,2.23,Wf7uumRk,0.561798,0.515464,0.537634,0.543478,0.077262,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.060826,0.007644,0.066395,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
216,2024-08-10,Austrália Nbl1 Leste,01:00,Newcastle Falcons F,Sutherland F,2.02,1.72,156.5,1.85,1.85,1.5,1.87,1.59,CtylasQk,0.495050,0.581395,0.540541,0.540541,0.076445,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,150.48,120.75,0.0,0.0,0.0,0.0,0.113440,0.000000,0.114445,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
217,2024-08-10,Austrália Nbl1 Sul,06:30,Ballarat,Eltham,1.86,1.86,167.5,1.83,1.85,-1.5,2.02,1.99,jNnVMQSI,0.537634,0.537634,0.546448,0.540541,0.075269,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,148.52,0.00,0.0,0.0,0.0,0.0,0.000000,0.007686,0.010580,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
218,2024-08-10,Austrália Nbl1 Sul,03:00,Keilor Thunder F,Waverley F,1.64,2.17,148.5,1.87,1.79,-2.5,1.84,2.08,MseEQ8Tg,0.609756,0.460829,0.534759,0.558659,0.070586,0.0,0.0,NaN,NaN,NaN,NaN,NaN,164.018,32.818647,0.200092,0.0,0.0,210.24,165.60,0.0,0.0,0.0,0.0,0.196728,0.030912,0.086585,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
219,2024-08-10,Austrália Nbl1 Central,07:30,South Adelaide Panthers,Forestville Eagles,1.86,1.86,173.5,1.86,1.80,-1.5,1.99,2.03,WYV4S9sp,0.537634,0.537634,0.537634,0.555556,0.075269,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,164.43,193.98,0.0,0.0,0.0,0.0,0.000000,0.023184,0.014072,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0
220,2024-08-10,Austrália Nbl1 Central,05:00,Central Districts Lions F,Forestville Eagles F,1.70,1.98,142.5,1.83,1.83,-2.5,1.94,2.45,zwu0feeK,0.588235,0.505051,0.546448,0.546448,0.093286,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,126.54,0.00,0.0,0.0,0.0,0.0,0.107603,0.000000,0.164294,0.0,0.0,inf,0.0,0.0,0.0,0.0,0.0,inf,0.0,0.0,0.0


## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,04:30,Austrália Nbl1 Leste,Canberra,Maitland M.,171.5,1.86,1.0000,Over
1,06:30,Austrália Nbl1 Sul,Ballarat,Eltham,167.5,1.83,0.9999,Over
2,03:00,Austrália Nbl1 Sul,Keilor Thunder F,Waverley F,148.5,1.87,0.9999,Over
3,07:30,Austrália Nbl1 Central,South Adelaide Panthers,Forestville Eagles,173.5,1.86,1.0000,Over
4,05:00,Austrália Nbl1 Central,Central Districts Lions F,Forestville Eagles F,142.5,1.83,1.0000,Over
